In [4]:
import os
nb_path = os.path.abspath("")
from sys import path
from os.path import dirname

path.insert(0,  dirname(nb_path))

nb_path

'c:\\Users\\ethan\\coding_projects\\pypreql\\experiments'

In [5]:
import pandas as pd
from preql import Executor, Dialects
from preql.core.models import Environment
from sqlalchemy import create_engine
from preql.core.models import Datasource, Concept, ColumnAssignment, Grain, Function
from preql.core.enums import DataType, Purpose, FunctionType
from os.path import dirname
from pathlib import PurePath
from preql.parsing.render import render_environment
import os


def create_dimension(exec:Executor, key:str, cols:list[str], name:str):
    exec.execute_raw_sql(f"CREATE SEQUENCE seq_{name} START 1;")
    exec.execute_raw_sql(f"""create table dim_{name} as 
                         SELECT passengerid, name FROM raw_data

""")
    
    print("DIM CREATED")
    
    
def create_fact(exec:Executor, dims:list[str] = None, include:list[str] = None):
    exec.execute_raw_sql(f"""create table fact_passenger as 
                         SELECT 
                         row_number() OVER () as fact_key,
                         passengerid,
                         survived 
                         
                         FROM raw_data
                         """)


def setup_normalized_engine() -> Executor:
    engine = create_engine(r"duckdb:///:memory:", future=True)
    csv = PurePath(os.path.abspath("")) / "train.csv"
    df = pd.read_csv(csv)
    output = Executor(engine=engine, dialect=Dialects.DUCK_DB)

    output.execute_raw_sql("CREATE TABLE raw_data AS SELECT * FROM df")
    create_dimension(output, 'name', ['age'], 'passenger')

    create_fact(output, ['passenger'])
    return output




def setup_titanic_distributed(env: Environment):
    namespace = "passenger"
    id = Concept(
        name="id", namespace=namespace, datatype=DataType.INTEGER, purpose=Purpose.KEY
    )
    age = Concept(
        name="age",
        namespace=namespace,
        datatype=DataType.INTEGER,
        purpose=Purpose.PROPERTY,
        keys=[id],
    )

    name = Concept(
        name="name",
        namespace=namespace,
        datatype=DataType.STRING,
        purpose=Purpose.PROPERTY,
        keys=[id],
    )

    pclass = Concept(
        name="class",
        namespace=namespace,
        purpose=Purpose.PROPERTY,
        datatype=DataType.INTEGER,
        keys=[id],
    )
    survived = Concept(
        name="survived",
        namespace=namespace,
        purpose=Purpose.PROPERTY,
        datatype=DataType.BOOL,
        keys=[id],
    )
    fare = Concept(
        name="fare",
        namespace=namespace,
        purpose=Purpose.PROPERTY,
        datatype=DataType.FLOAT,
        keys=[id],
    )
    embarked = Concept(
        name="embarked",
        namespace=namespace,
        purpose=Purpose.PROPERTY,
        datatype=DataType.INTEGER,
        keys=[id],
    )
    cabin = Concept(
        name="cabin",
        namespace=namespace,
        purpose=Purpose.PROPERTY,
        datatype=DataType.STRING,
        keys=[id],
    )
    last_name = Concept(
        name="last_name",
        namespace=namespace,
        purpose=Purpose.PROPERTY,
        datatype=DataType.STRING,
        keys=[id],
        lineage=Function(
            operator=FunctionType.INDEX_ACCESS,
            arguments=[
                Function(
                    operator=FunctionType.SPLIT,
                    arguments=[name, ','],
                    output_datatype=DataType.ARRAY,
                    output_purpose=Purpose.PROPERTY,
                    arg_count = 2,
                ),
                1,
            ],
            output_datatype=DataType.STRING,
            output_purpose=Purpose.PROPERTY,
            arg_count=2,
        ),
    )
    for x in [id, age, survived, name, pclass, fare, cabin, embarked, last_name]:
        env.add_concept(x)

    env.add_datasource(
        Datasource(
            identifier="dim_passenger",
            address="dim_passenger",
            columns=[
                ColumnAssignment(alias="id", concept=id),
                ColumnAssignment(alias="age", concept=age),
                ColumnAssignment(alias="name", concept=name)
                # ColumnAssignment(alias="survived", concept=survived),
                # ColumnAssignment(alias="pclass", concept=pclass),
                # ColumnAssignment(alias="name", concept=name),
                # ColumnAssignment(alias="fare", concept=fare),
                # ColumnAssignment(alias="cabin", concept=cabin),
                # ColumnAssignment(alias="embarked", concept=embarked),
            ],
            grain=Grain(components=[id]),
        ),
    )

    env.add_datasource(
        Datasource(
            identifier="fact_titanic",
            address="fact_titanic",
            columns=[
                ColumnAssignment(alias="passenger_id", concept=id),
                # ColumnAssignment(alias="age", concept=age),
                ColumnAssignment(alias="survived", concept=survived),
                # ColumnAssignment(alias="pclass", concept=pclass),
                # ColumnAssignment(alias="name", concept=name),
                ColumnAssignment(alias="fare", concept=fare),
                # ColumnAssignment(alias="cabin", concept=cabin),
                # ColumnAssignment(alias="embarked", concept=embarked),
            ],
            grain=Grain(components=[id]),
        ),
    )
    
    return env




In [8]:
env = Environment()
from logging import DEBUG, StreamHandler
from preql.constants import logger


if len(logger.handlers) == 0:
    logger.addHandler(StreamHandler())
# logger.setLevel(DEBUG)

executor = setup_normalized_engine()

titanic_env = setup_titanic_distributed(env)

executor.environment = titanic_env

generated = executor.generate_sql(
    """
select passenger.survived, passenger.name, passenger.last_name;
    """
)

print(generated[0])

[QUERY BUILD] getting source datasource for query with output ['passenger.survived<>', 'passenger.name<>', 'passenger.last_name<>']
[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.survived<>', 'passenger.name<>', 'passenger.last_name<>']
[CONCEPT DETAIL] For passenger.survived, have local optional ['passenger.name<passenger.id>', 'passenger.last_name<passenger.id>']
GEN SELECT NODE HAS ['passenger.survived', 'passenger.name', 'passenger.last_name'] and ['passenger.name']
CHECKING dim_passenger
CHECKING fact_titanic
	[CONCEPT DETAIL] Beginning sourcing loop for ['passenger.id<passenger.id>', 'passenger.age<>', 'passenger.name<>']
	[CONCEPT DETAIL] For passenger.id, have local optional ['passenger.age<passenger.id>', 'passenger.name<passenger.id>']
GEN SELECT NODE HAS ['passenger.id', 'passenger.age', 'passenger.name'] and ['passenger.age', 'passenger.name']
	[CONCEPT DETAIL - SELECT NODE] found direct select from location='dim_passenger' for ['passenger.id', 'passenger.age', 'p

DIM CREATED


	[CONCEPT DETAIL] have all concepts, have ['passenger.id', 'passenger.survived', 'passenger.fare'] from [SelectNode<passenger.fare,passenger.id,passenger.survived>] checking for single connected graph
	[CONCEPT DETAIL] Graph analysis: 1 subgraphs found
	[CONCEPT DETAIL] One fully connected subgraph returned, sourcing ['passenger.id', 'passenger.survived', 'passenger.fare'] successful.
	[CONCEPT DETAIL - MERGE NODE] Merge node has only one parent with the same outputs as this merge node, dropping merge node 
[CONCEPT DETAIL - MERGE NODE] Merge node has 2 parents, starting merge
[CONCEPT DETAIL - MERGE NODE] potential merge keys ['passenger.id', 'passenger.age', 'passenger.name'] for dim_passenger_at_passenger_id
[CONCEPT DETAIL - MERGE NODE] potential merge keys ['passenger.id', 'passenger.survived', 'passenger.fare'] for fact_titanic_at_passenger_id
[CONCEPT DETAIL] finished a loop iteration looking for ['passenger.survived', 'passenger.name', 'passenger.last_name'] from [MergeNode<pas


WITH 
serval as (
SELECT
    local_fact_titanic."passenger_id" as "passenger_id",
    local_fact_titanic."survived" as "passenger_survived",
    local_fact_titanic."fare" as "passenger_fare"
FROM
    fact_titanic as local_fact_titanic
),
albatross as (
SELECT
    local_dim_passenger."id" as "passenger_id",
    local_dim_passenger."age" as "passenger_age",
    local_dim_passenger."name" as "passenger_name"
FROM
    dim_passenger as local_dim_passenger
),
avocet as (
SELECT
    albatross."passenger_name" as "passenger_name",
    serval."passenger_survived" as "passenger_survived"
FROM
    albatross as albatross

LEFT OUTER JOIN serval on albatross."passenger_id" = serval."passenger_id"

),
dynamic as (
SELECT
    STRING_SPLIT( avocet."passenger_name" , ',' )[1] as "passenger_last_name",
    avocet."passenger_survived" as "passenger_survived",
    avocet."passenger_name" as "passenger_name"
FROM
    avocet as avocet
),
helpless as (
SELECT
    dynamic."passenger_survived" as "passenger_s